In [1]:
!pip install langchain langchain_community sentence-transformers faiss-cpu transformers torch accelerate

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/41.3 MB ? eta -:--:--
    --------------------------------------- 0.5/41.3 MB 728.2 kB/s eta 0:00:56
    --------------------------------------- 0.8/41.3 MB 905.5 kB/s eta 0:00:45
   - -------------------------------------- 1.6/41.3 MB 1.4 MB/s eta 0:00:28
   - -------------------------------------- 1.6/41.3 MB 1.4 MB/s eta 0

    sphinx-rtd-theme (>=0.1.10b0S) ; extra == 'doc'
                     ~~~~~~~~~~~^
    sphinx-rtd-theme (>=0.1.10b0S) ; extra == 'doc'
                     ~~~~~~~~~~~^
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepsort 0.0.5 requires scikit_learn==1.1.3, but you have scikit-learn 1.6.1 which is incompatible.
deepsort 0.0.5 requires scipy==1.9.3, but you have scipy 1.15.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
openai-whisper 20230314 requires tiktoken==0.3.1, but you have tiktoken 0.3.3 which is incompatible.
p5 0.8.4 requires Pillow==9.0.1, but you have pillow 10.0.1 which is incompatible.
pyannote-database 5.1.0 requires typer>=0.12.1, but you have typer 0.7.0 which is

In [2]:
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

c:\Users\aiden\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
class Config:
    SEED: int = 42
    SAMPLE: int = 10000
    BASE_DATA_PATH: str = '../data/'
    BASE_OUTPUT_PATH: str = '../output/'

In [4]:
file_path = f'{Config.BASE_DATA_PATH}/arXiv_scientific dataset.csv'

df = pd.read_csv(file_path)

In [5]:
df.columns

Index(['id', 'title', 'category', 'category_code', 'published_date',
       'updated_date', 'authors', 'first_author', 'summary',
       'summary_word_count'],
      dtype='object')

In [6]:
df = df.sample(Config.SAMPLE, random_state=Config.SEED)

In [7]:
len(df)

10000

In [8]:
df.columns

Index(['id', 'title', 'category', 'category_code', 'published_date',
       'updated_date', 'authors', 'first_author', 'summary',
       'summary_word_count'],
      dtype='object')

In [9]:
docs = [
    f"""title: {title}
author: {authors}
publish_date: {published_date}
summary: {summary}"""
    for title, authors, published_date, summary in zip(df["title"], df["authors"], df["published_date"], df["summary"])
]

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = []
for d in docs:
    texts.extend(splitter.split_text(d))

In [10]:
print(f'{len(docs)=}')
print(f'{len(texts)=}')

len(docs)=10000
len(texts)=10000


In [11]:
import pprint as pp

pp.pprint(docs[0])

('title: Machine Learning and the Future of Realism\n'
 "author: ['Giles Hooker', 'Cliff Hooker']\n"
 'publish_date: 4/15/17\n'
 'summary: The preceding three decades have seen the emergence, rise, and '
 'proliferation\n'
 'of machine learning (ML). From half-recognised beginnings in perceptrons,\n'
 'neural nets, and decision trees, algorithms that extract correlations (that '
 'is,\n'
 'patterns) from a set of data points have broken free from their origin in\n'
 'computational cognition to embrace all forms of problem solving, from voice\n'
 'recognition to medical diagnosis to automated scientific research and\n'
 'driverless cars, and it is now widely opined that the real industrial\n'
 'revolution lies less in mobile phone and similar than in the maturation and\n'
 'universal application of ML. Among the consequences just might be the '
 'triumph\n'
 'of anti-realism over realism.')


In [12]:
embed = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(docs, embed)
vectorstore.save_local(f"{Config.BASE_OUTPUT_PATH}/faiss_index")

C:\Users\aiden\AppData\Local\Temp\ipykernel_21152\1965970792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [13]:
hf_pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device_map="cpu"  # CPU by default; or set device_map={"": 0} for GPU
)
llm = HuggingFacePipeline(pipeline=hf_pipe)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

c:\Users\aiden\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aiden\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

(…)a5b18a05535c9e14c7a355904270e15b0945ea86:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

C:\Users\aiden\AppData\Local\Temp\ipykernel_21152\1499822159.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipe)


In [14]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,  # <-- return docs in output
    verbose=True,                  # <-- print chain steps (optional)
)

In [15]:
res = qa({"query": "Who are author of Machine Learning and the Future of Realism?"})
print("🏷️ Answer:\n", res["result"], "\n")
print("📄 Retrieved documents:")
for i, doc in enumerate(res["source_documents"], 1):
    # doc.page_content is the text, doc.metadata might have your CSV row info
    print(f"\n----- Doc #{i} -----")
    print(doc.metadata)
    print(doc.page_content)

C:\Users\aiden\AppData\Local\Temp\ipykernel_21152\4218667418.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa({"query": "Who are author of Machine Learning and the Future of Realism?"})
Token indices sequence length is longer than the specified maximum sequence length for this model (1794 > 512). Running this sequence through the model will result in indexing errors




> Entering new RetrievalQA chain...


c:\Users\aiden\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



> Finished chain.
🏷️ Answer:
 Giles Hooker 

📄 Retrieved documents:

----- Doc #1 -----
{}
title: Machine Learning and the Future of Realism
author: ['Giles Hooker', 'Cliff Hooker']
publish_date: 4/15/17
summary: The preceding three decades have seen the emergence, rise, and proliferation
of machine learning (ML). From half-recognised beginnings in perceptrons,
neural nets, and decision trees, algorithms that extract correlations (that is,
patterns) from a set of data points have broken free from their origin in
computational cognition to embrace all forms of problem solving, from voice
recognition to medical diagnosis to automated scientific research and
driverless cars, and it is now widely opined that the real industrial
revolution lies less in mobile phone and similar than in the maturation and
universal application of ML. Among the consequences just might be the triumph
of anti-realism over realism.

----- Doc #2 -----
{}
title: The role of prior information and computational pow

In [16]:
res['result']

'Giles Hooker'